# Understanding binary Goppa decoding
This notebook follows the short [introduction to binary Goppa decoding](https://eprint.iacr.org/2022/473.pdf) written by DJB.

# Preliminaries

## The Bernoulli rule
Let $K$ be some field, and let $K[x]$ denote the ring of polynomial with coefficients in the field $K$. Let $f, g \in K[x]$ be two polynomials. The Bernoulli rule is concerned with evaluating the rational function $f/g$ at some point $\alpha$ where $f(\alpha) = g(\alpha) = 0$:

> (**The Bernoulli rule**) If $\alpha \in K$ is such that $f(\alpha) = g(\alpha) = 0$ and $g^\prime(\alpha) \neq 0$, then $(f/g)(\alpha) = \frac{f^\prime(\alpha)}{g^\prime(\alpha)}$, where $f^\prime$ is the derivative of $f$

This is also commonly referred to as the L'Hôpital's rule, although [the origin of this rule traces back to Johann Bernoulli](https://en.wikipedia.org/wiki/L%27H%C3%B4pital%27s_rule).

# Polynomial interpolation
In this section we show that a degree $n-1$ polynomial can be uniquely determined using $n$ distinct points, then present the algorithm to compute such points.

Let $(\alpha_1, \alpha_2, \ldots, \alpha_n) \in K$ be $n$ distinct points, and $r_1, r_2, \ldots, r_n$ be $n$ (not necessarily distinct) values, then there exists a polynomial $f$ such that $\deg(f) < n$ and $f(\alpha_i) = r_i$ for all $1 \leq i \leq n$. $f$ can be explicitly expressed by the formula:

$$
f = \sum_{i = 1}^{n}  \left\lparen r_i
    \prod_{j \neq i} \frac{(x - \alpha_j)}{(\alpha_i - \alpha_j)}
\right\rparen
$$

We can do a quick sanity check: $\prod_{j \neq i} \frac{(x - \alpha_j)}{(\alpha_i - \alpha_j)}$ is a degree $n - 1$ polynomial, so $f$ is a sum of $n$ of degree $n-1$ polynomial, which makes $\deg(f) \leq n-1$. it is straightforward to verify that $f(\alpha_i) = r_i$ for all $1 \leq i \leq n$.

If there exists another polynomial $g$ such that $\deg(g) < n$, $g \neq f$, and $g(\alpha_i) = f(\alpha_i) = r_i$ for all $1 \leq i \leq n$, then $g - f$ is a non-zero polynomial whose degree is less than $n$ but has $n$ distinct roots at $\alpha_1, \alpha_2, \ldots, \alpha_n$, which contradicts the [fundamental theorem of algebra](https://en.wikipedia.org/wiki/Fundamental_theorem_of_algebra). Therefore, $f$ must be unique.

In [1]:
# TODO: use sympy (or don't!) to implement polynomial interpolation
def interpolate(points):
    pass

# Best Approximant Theorem
Suppose $A, B \in K[x]$ are polynomials such that $\deg(B) < \deg(A)$. The best approximant theorem tries to find polynomials $a, b \in K[x]$ of much lower degree $t$ such that $aB - bA$ has low degree. $a, b$ are called approximant because if $aB - bA$ has low degree, then $\frac{a}{b}$ is a good approximation of $\frac{A}{B}$.

**Best approximant theorem.** Let $K$ be a field and $K[x]$ denote the ring of polynomials. For every $A, B \in K[x]$ such that $\deg(A) > \deg(B)$ and non-zero integer $t \geq 0$ such that $2t < \deg(A)$, there exists polynomials $a, b \in K[x]$ such that:
1. $\deg(a) \leq t, \deg(b) < t$
1. $\gcd(a, b) = 1$
1. $\deg(aB - bA) < \deg(A) - t$ 

Furthermore, if there also exist $c, d \in K[x]$ such that $\deg(c) \leq t$ and $\deg(cB - dA) < \deg(A) - t$, then $(c, d) = (\lambda a, \lambda b)$ for some $\lambda \in K[x]$.

The best approximant theorem states that there exists a unique pair of lower-degree co-prime polynomials $a, b$ that can approximate $A, B$ to the specified degree, and any other pairs that can approximate $A, B$ to that degree must be multiples of $a, b$.

## Proof of best approximant theorem
Denote $\deg(A)$ by $n$.

Because $\deg(a) \leq t$, $a$ can be represented using $t + 1$ coefficients. Similarly $b$ can be represented using $t$ coefficients. Thus, the set of coefficients of $a, b$ together form a $2t+1$ dimensional vector space. On the other hand, $\deg(aB + bA) = \max(\deg(aB), \deg(bA))$. Since $\deg(aB) = \deg(a) + \deg(B) < n + t$ and $\deg(bA) = \deg(b) + \deg(A) < n + t$, we know that $\deg(aB + bA) < n + t$. Thus we can write $aB + bA$ as:

$$
aB + bA = k_{n + t - 1} x^{n + t - 1} + k_{n + t - 2} x^{n + t - 2} + \ldots + k_1x + k_0
$$

It is easy to show that because $(a, b) \mapsto aB + bA$ is a linear map with respect to $(a, b)$, the following map is also a linear map (**is it really obvious though?**):

$$
(a_0, a_1, \ldots, a_t, b_0, b_1, \ldots, b_{t-1}) \mapsto (k_{n+t-1}, k_{n+t-2}, \ldots, k_{n-t})
$$

Observe that the LHS is a $2t+1$-dimensional space, while the RHS is a $2t$-dimensional space. By the **rank-nullity theorem**, we know that this mapping has non-zero roots, which means that $aB - bA$ has degree at most $n - t - 1$. Since it is easy to write out $aB - bA$ explicitly in terms of $(a, b)$, we can find such non-zero $(a, b)$ using Gaussian elimination on a $K^{2t \times (2t+1)}$ matrix.

If $(a, b)$ is not co-prime, then we can find their GCD (**How**) and replace $(a, b)$ with $(\frac{a}{\gcd(a, b)}, \frac{b}{\gcd(a,b)})$. The degree of $aB - bA$ will decrease by the degree of the GCD, so it will still remain in the bound.

Now suppose $(a, b)$ have been found, and $(c, d)$ is another pair of polynomials such that $\deg(c) \leq t, \deg(d) < t, \deg(cB - dA) < n - t$, then observe the following:

$$
\det\left(\begin{bmatrix}
c & a \\
cB - dA & aB - bA
\end{bmatrix}\right) = c(aB - bA) - a(cB - dA) = (ad - cb)A
$$

Because $\deg(aB - bA) < n - t$ and $\deg(c) < t$, $\deg(c(aB - bA)) < n$. Similarly, $\deg(a(cB - dA)) < n$, which means that $\deg(c(aB - bA) - a(cB - dA)) < n$. However, $\deg((ad - cb)A) \geq \deg(A) = n$ if $ad - cb \neq 0$, so it must be that $ad - cb = 0$, which implies $ad = cb$. Since $\gcd(a, b) = 1$, it must be that $a \mid c$. From here it's easy to show that $(c, d) = (\lambda a, \lambda b)$ for some $\lambda \in K[x]$.

In [2]:
# TODO: implement best approximant theorem
def best_approximant(A, B):
    pass

# Interpolation with error
Let $n, t$ be two non-negative integers such that $2t < n$. This section describes how one can recover a polynomial of degree less than $n - 2t$ using $n$ points with up to $t$ errors.

Let $K$ be a field and $K[x]$ be the ring of polynomials. Let $\alpha_1, \alpha_2, \ldots, \alpha_n \in K$ be $n$ distinct values. Let $f \in K[x]$ be a polynomial such that $\deg(f) < n - 2t$, and let $f(\alpha_1), f(\alpha_2), \ldots, f(\alpha_n) \in K$ be the evaluation of $f$ at these distinct values. Let $r_1, r_2, \ldots, r_n$ be such that the Hamming distance between $(r_1, r_2, \ldots, r_n)$ and $(f(\alpha_1), f(\alpha_2), \ldots, f(\alpha_n))$ is at most $t$. Denote $(\alpha_1, \alpha_2, \ldots, \alpha_n)$ by $\vec{\alpha}$, $(f(\alpha_1), f(\alpha_2), \ldots, f(\alpha_n))$ by $f(\vec{\alpha})$, and $(r_1, r_2, \ldots, r_n)$ by $\vec{r}$. We can use the following routine to recover $f$ using only $\vec{\alpha}$ and $\vec{r}$:
1. Compute $A = \prod_{i = 1}^n (x - \alpha_i)$. It's easy to see that $\deg(A) = n$.
1. Apply Lagrange interpolation to $\vec{\alpha}, \vec{r}$, which returns polynomial $B$. From previous theorem we know $\deg(B) < n$
1. Use the [best approximant theorem](#best-approximant-theorem) to compute polynomials $a, b \in K[x]$ such that $\deg(a) \leq t$ and $\deg(aB - bA) < n - t$
1. Compute $\hat{f} = B - bA/a$, which is the output.

***Proof***:

Let $S = \{i \in \{1, 2, \ldots, n\} \mid r_i = f(\alpha_i) \}$ be the set of points on which $B$ and $f$ agree, then $A$ can be expressed as two products:

$$
A = \left(\prod_{i \in S}(x - \alpha_i)\right)\left(\prod_{i \not\in S}(x - \alpha_i)\right)
$$

Denote $\prod_{i\in S}(x - \alpha_i)$ by $E$ and $\prod_{i \not\in S}(x - \alpha_i)$ by $c$, then $A = Ec$.

Because for every $i \in S$, $B(\alpha_i) = f(\alpha_i)$, which implies that $(x - \alpha_i)l$ divides $B - f$ in $K[x]$, we know that $\prod_{i \in S}(x - \alpha_i)$ divides $B - f$ in $K[x]$. In other words, there exists some polynomial $d \in K[x]$ such that:

$$Ed = B - f$$

Multiply both sides of $A = Ec$ by $d$:

$$dA = dEc = (B - f)c$$

Which re-arranges to a familiar form:

$$cB - dA = cf$$

The degree of $c$ is at most $t$ because there are at most $t$ errors. The degree of $f$ is less than $n - 2t$, so the degree of $cf$ is less than $n - t$. Furthermore, we have $\deg(A) = n$, $\deg(B) \leq n - 1 < n$, which means that **(c, d) are approximants of (A, B)**. By the [Best Approximant Theorem](#best-approximant-theorem), $(c, d)$ must be a non-zero multiple of the best approximant $(a, b)$. In other words, there exists $\lambda \in K[x]$ such that $(c, d) = \lambda(a, b)$.

This means that $a$ divides $c$ in $K[x]$. Since $A = Ec$, it naturally follows that $a$ divides $A$ in $K[x]$, so $A / a$ is well-defined in $K[x]$.

Finally, divide both sides of $cB - dA = cf$  by $c$:

$$
B - dA/c = f
$$

Here $d/c = b/a$, so the equation above is equivalent to:

$$
f = B - bA/a
$$

In [ ]:
# TODO: implement interpolation with error
def interpolate_with_error(points):
    pass

## Some additional comments on [interpolation with error](#interpolation-with-error)
Interpolation with error is the basis of Reed-Soloman code. In fact, we have everything needed to describe Reed-Soloman code in full:

- Each instance of a Reed-Soloman code is parameterized by:
    - some finite field $\mathbb{K}$, which induces a polynomial ring \mathbb{K}[x]$.
    - non-negative integers $n, t$ such that $2t < n$. $n$ is the codeword size (each codeword is $n$ field element), and $t$ is the error-correcting capacity
    - $n$ distinct field elements $\alpha_1, \alpha_2, \ldots, \alpha_n \in \mathbb{K}$ 
- The message space is $\mathcal{M} = \mathbb{K}^{n - 2t}$. To encode a message $m = (m_0, m_1, \ldots, m_{n - 2t - 1})$, first define a polynomial $f = m_0 + m_1 x + \ldots + m_{n - 2t - 1}x^{n - 2t - 1} \in \mathbb{K}[x]$, then evaluate $f$ at each of $\alpha_1, \alpha_2, \ldots, \alpha_n$: $f(\vec{\alpha}) = (f(\alpha_1), f(\alpha_2), \ldots, f(\alpha_n)) \in \mathbb{K}^n$. Output $f(\vec{\alpha})$ is the codeword.
- The decoder receives some $\vec{r} = (r_1, r_2, \ldots, r_n) \in \mathbb{K}^n$ as the input. If $\vec{r}$ contains no more than $t$ errors, then applying the "Interpolation with error" algorithm allows the decoder to recover $f$. From here it's trivial to recover the message.

# Binary Goppa Code
Here we provide an incomplete introduction to the Binary Goppa Code with an emphasis on the decoding procedure.

Each instance of a Binary Goppa code is parameterized by:
- Base field $\mathbb{K}$ that contains $\mathbb{F}_2$ ($\mathbb{K}$ is typically $\mathbb{F}_{2^m}$)
- Non-negative integers $n, t$ such that $2t < n$
- $n$ distinct field elements $\alpha_1, \alpha_2, \ldots, \alpha_n$
- An irreducible square-free polynomial $g \in \mathbb{K}[x]$ of degree $t$ such that $g(\alpha_i) \neq 0$ for all $1 \leq i \leq n$, or equivalently $\gcd(A, g) = 1$ where $A = \prod_{i=1}^n (x-\alpha_i)$

Let $A = \prod_{i=1}^n(x - \alpha_i)$, the set of codewords is as follows:

$$
\mathcal{C} = \left\{
    \mathbf{c} = (c_1, c_2, \ldots, c_n) \in \mathbb{F}_2^n 
    \mid \sum_{i=1}^n \frac{c_i A}{x - \alpha_i} \equiv 0 \mod g
\right\}
$$

For the purpose of understanding the classic McEliece cryptosystem, we don't need to know the encoding routine. This is because the Goppa decoding routine actually directly recovers the codeword/error.